<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ССЫЛКА НА ОРИГИНАЛ:

https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/unit3/unit3.ipynb#scrollTo=ykJiGevCMVc5

@huggingface

@simoninithomas

# # Блок 3: Изучение глубокого Q-обучения с помощью игр Atari 👾 с использованием RL Baselines3 Zoo

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/thumbnail.jpg" alt="Unit 3 Thumbnail" width="50%">

В этом блокноте **вы будете тренировать агента глубокого Q-обучения**, играя в "Космические захватчики""(Space Invaders), используя [RL Baselines 3 Bio](https://github.com/DLR-RM/rl-baselines3-zoo), система обучения, основанная на[Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/) которая предоставляет сценарии для обучения, оценки агентов, настройки гиперпараметров, отображения результатов и записи видео.

Мы будим использовать [RL-Baselines-3 Zoo integration, a vanilla version of Deep Q-Learning](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) без расширений, таких как Double-DQN, Dueling-DQN, and Prioritized Experience Replay.

⬇️Вот пример того, чего **вы достигнете** ⬇️



In [1]:
%%html
<video controls autoplay><source src="https://huggingface.co/ThomasSimonini/ppo-SpaceInvadersNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

### 🎮 Окружающая среда: 

- SpacesInvadersNoFrameskip-v4 

### 📚 RL-библиотека: 

- [RL-Baselines3-Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

## Цели этой записной книжки 🏆
В конце вы:
- Сможете глубже понять **как работает RL Baselines3 Zoo**.
- Будете иметь возможность **отправить своего обученного агента и код в центр** с хорошим видеоповтором и оценочным баллом 🔥 .


## Этот блокнот взята из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%" />

## Предварительные условия 🏗️
Прежде чем погрузиться в блокнот, вам нужно:

🔲  📚 **[Изучить глубокое Q-обучение, прочитав блок 3](https://huggingface.co/deep-rl-course/unit3/introduction)**  🤗 

# Давайте обучим агента глубокого Q-обучения игре Atari "Space Invaders" и загрузим ее в Хаб.

Чтобы проверить это практическое руководство для процесса сертификации, вам нужно отправить свою обученную модель в центр и **получить результат >= 200**.

Чтобы узнать свой результат, перейдите в таблицу лидеров и найдите свою модель, **результат = mean_reward - std of reward**

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Совет 💡
Лучше запустить этот colab в копии на вашем Google Диске, чтобы **в случае истечения времени ожидания** у вас все еще была сохраненная записная книжка на вашем Google Диске, и вам не нужно было заполнять все с нуля.

Для этого вы можете либо нажать `Ctrl + S`, либо `File > Save a copy in Google Drive.`

Кроме того, мы собираемся **тренировать его в течение 90 минут с количеством шагов 1 м**. Набрав `!nvidia-smi`, вы узнаете, какой графический процессор вы используете.

И если вы хотите тренировать больше 10 миллионов шагов, это займет около 9 часов, что потенциально приведет к отключению Colab. В этом случае я рекомендую запустить это на вашем локальном компьютере (или где-нибудь еще). Просто нажмите на: `Файл> Загрузить`.

## Установка графического процессора 💪
- Чтобы **ускорить обучение агента, мы будем использовать графический процессор**. Чтобы сделать это, перейдите к `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="50%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="50%">

## Создайте виртуальный дисплей 🔽

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab **нам нужен виртуальный экран, чтобы иметь возможность отображать среду** (и, таким образом, записывать кадры).

Следовательно, следующая ячейка установит библиотеки, создаст и запустит виртуальный экран 🖥

In [2]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [3]:
%%capture
# Дополнительные зависимости для RL Baselines3 Zoo
!apt-get install swig cmake freeglut3-dev 

In [4]:
!pip install pyglet==1.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.7 MB/s eta 0:00:00


In [6]:
# Виртуальный дисплей
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Клонирование репозитория RL-Baselines 3 Zoo 📚
Теперь можно установить непосредственно из пакета python `pip install rl_zoo3`, но поскольку мы хотим **полную установку с дополнительными средами и зависимостями**, мы собираемся клонировать репозиторий `RL-Baselines3-Zoo` и установить из исходного кода.

In [7]:
!git clone https://github.com/DLR-RM/rl-baselines3-zoo

Cloning into 'rl-baselines3-zoo'...
remote: Enumerating objects: 4907, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 4907 (delta 119), reused 133 (delta 76), pack-reused 4707
Receiving objects: 100% (4907/4907), 3.07 MiB | 9.23 MiB/s, done.
Resolving deltas: 100% (3242/3242), done.


## Установка зависимостей 🔽
Теперь мы можем установить зависимости, необходимые RL-Baselines 3 Zoo (это может занять 5 минут ⏲ )

In [8]:
%cd /content/rl-baselines3-zoo/

/content/rl-baselines3-zoo


In [ ]:
!pip install -r requirements.txt

## Обучение агента Deep Q-Learning играть в Space Invaders👾

Чтобы обучить агента с помощью RL-Baselines 3-Zoo, нам просто нужно сделать две вещи:

1. Определить гиперпараметры в `rl-baselines3-zoo/hyperparams/dqn.yml`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/hyperparameters.png" alt="DQN Hyperparameters">


Из чего видно, что:
- Мы используем `оболочку Atari`, которая предварительно обрабатывает входные данные (уменьшение кадра, перевод в оттенки серого, создание стека из 4 кадров)
- Мы используем `CnnPolicy`, мы используем сверточные слои для обработки кадров
- Мы тренируем его в течение 10 миллионов "n_timesteps"
- Объем памяти (воспроизведение опыта) равен 100000, то есть количеству шагов опыта, которые вы сохранили, чтобы снова обучить своего агента.

💡 Мой совет - **сократить время тренировки до 1 м,** что займет около 90 минут на P100. `!nvidia-smi` сообщит вам, какой графический процессор вы используете. Обучение при 10 миллионах шагов займет около 9 часов, что, вероятно, может привести к тайм-ауту Colab. Я рекомендую запустить это на вашем локальном компьютере (или где-нибудь еще). Просто нажмите на: `Файл> Загрузить`.

Что касается оптимизации гиперпараметров, мой совет - сосредоточиться на этих 3 гиперпараметрах:
- `learning_rate`
- `buffer_size (размер оперативной памяти)`
- `batch_size (размер пакета)`

В качестве хорошей практики вам необходимо **проверить документацию, чтобы понять, что делает каждый гиперпараметр**: https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters

2. Запустить `train.py` и сохраните модели в папке `logs` 📁

In [ ]:
# заполните пробелы
!python train.py --algo ________ --env SpaceInvadersNoFrameskip-v4  -f _________

#### Решение

In [ ]:
!python train.py --algo dqn  --env SpaceInvadersNoFrameskip-v4 -f logs/

Выходные данные были обрезаны до нескольких последних строк (5000).
|    learning_rate    | 0.0001   |
|    loss             | 0.00612  |
|    n_updates        | 27315    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.35e+03 |
|    ep_rew_mean      | 222      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 1140     |
|    fps              | 261      |
|    time_elapsed     | 802      |
|    total_timesteps  | 209916   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0335   |
|    n_updates        | 27478    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.36e+03 |
|    ep_rew_mean      | 224      |
|    exploration_rate | 0.01     |
| time/               |          |
|    episodes         | 1144     |
|    fps              

## Давайте оценим нашего агента 👀
- RL-Baselines 3-Zoo предоставляет `enjoy.py `, скрипт на python для оценки агента. В большинстве библиотек RL мы вызываем сценарий оценки `enjoy.py `.
- Давайте оценим его на 5000 временных шагах 🔥

In [ ]:
# заполните пробелы
!python enjoy.py  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps _________  --folder logs/

#### Решение

In [ ]:
!python enjoy.py  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps 5000  --folder logs/

## Опубликуйте обученную модель на хабе 🚀
Теперь, когда мы увидели, что получили хорошие результаты после обучения, мы можем опубликовать нашу обученную модель на хабе 🤗 с помощью одной строки кода.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/space-invaders-model.gif" alt="Space Invaders model" width="50%">

С помощью `rl_zoo3.push_to_hub.py ` **вы оцениваете, записываете повтор, генерируете карточку модели вашего агента и отправляете ее в центр**.

Таким образом:
- Вы можете **продемонстрировать свою работу** 🔥
- Вы можете **визуализировать, как ваш агент играет** 👀
- Вы можете **поделиться с сообществом агентом, которым могут пользоваться другие** 💾
- Вы можете **получить доступ к таблице лидеров 🏆 , чтобы увидеть, насколько хорошо работает ваш агент по сравнению с вашими одноклассниками** 👉  https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

Чтобы иметь возможность поделиться своей моделью с сообществом, необходимо выполнить еще три шага:

1️⃣ (если это еще не сделано) создайте учетную запись для HF ➡ https://huggingface.co/join

2️⃣ Войдите в систему, и сохраните свой токен аутентификации с веб-сайта Hugging Face.
- Создайте новый токен (https://huggingface.co/settings/tokens ) **с ролью записи**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token" width="30%">

- Скопируйте токен
- Запустите ячейку ниже и вставьте токен

Если вы не хотите использовать записную книжку Google Colabora Jupyter, вам нужно вместо этого использовать эту команду: `huggingface-cli login`

3️⃣ Теперь мы готовы направить нашего обученного агента в 🤗 Hub 🔥

Давай запустим push_to_hub.py файл для загрузки нашего обученного агента в концентратор.

`--repo-name `: название отчета

`-orga`: Имя пользователя HF

In [ ]:
# заполните пробелы
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name _____________________ -orga _____________________ -f logs/

#### Решение

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name dqn-SpaceInvadersNoFrameskip-v4  -orga Art-phys  -f logs/

#### УРА

Поздравляем 🥳 Вы только что обучили и загрузили свего первого агент глубокого Q-обучения с использованием RL-Baselines-3 Zoo. Приведенный выше скрипт должен был отобразить ссылку на репозиторий моделей, например https://huggingface.co/ThomasSimonini/dqn-SpaceInvadersNoFrameskip-v4 . Когда вы переходите по этой ссылке, вы можете:

- Смотреть **предварительный просмотр видео вашего агента**.
- Нажать "Файлы и версии", чтобы просмотреть все файлы в репозитории.
- Нажать "Use in stable-baselines3", чтобы получить фрагмент кода, который показывает, как загрузить модель.
- Посмотреть карточку модели (`README.md ` файл), которая дает описание модели и гиперпараметров, которые вы использовали.

Под капотом концентратор использует репозитории на основе git (не волнуйтесь, если вы не знаете, что такое git), что означает, что вы можете обновлять модель новыми версиями по мере экспериментов и улучшения вашего агента.

****Сравните результаты ваших агентов со своими одноклассниками** с помощью [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) 🏆

## Загрузите мощную обученную модель 🔥
- Команда Stable-Baselines 3 загрузила **более 150 обученных агентов глубокого подкрепления на Хаб**.

Вы можете найти их здесь: 👉 https://huggingface.co/sb3

Несколько примеров:
- Asteroids(Астероиды): https://huggingface.co/sb3/dqn-AsteroidsNoFrameskip-v4
- Beam Rider(Луч всадника): https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4
- Breakout(Прорыв): https://huggingface.co/sb3/dqn-BreakoutNoFrameskip-v4
- Road Runner(Дорожный бегун): https://huggingface.co/sb3/dqn-RoadRunnerNoFrameskip-v4

Давайте загрузим агента, играющего в Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

1. Мы загружаем модель с помощью `rl_bios 3.load_from_hub` и помещаем ее в новую папку, которую мы можем назвать `rl_trained`

In [ ]:
# Загрузите модель и сохраните ее в папке logs/
!python -m rl_zoo3.load_from_hub --algo dqn --env BeamRiderNoFrameskip-v4 -orga sb3 -f rl_trained/

2. Давайте оценим, на 5000 временных шагах

In [ ]:
!python enjoy.py --algo dqn --env BeamRiderNoFrameskip-v4 -n 5000  -f rl_trained/

Почему бы не попробовать обучить своего собственного **Агента глубокого Q-обучения, играть в BeamRiderNoFrameskip-v4? 🏆.**

Если вы хотите попробовать, проверьте https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4#hyperparameters **в карточке модели у вас есть гиперпараметры обученного агента.**

Но поиск гиперпараметров может оказаться непростой задачей. К счастью, в следующем разделе мы увидим, как мы можем **использовать `Optuna` для оптимизации гиперпараметров 🔥 .**

## Дополнение 🏆
Лучший способ научиться **- это попробовать что-то самостоятельно**!

В [Leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) вы найдете своих агентов. Сможете ли вы добраться до вершины?

Вот список сред, в которых вы можете попробовать обучить своего агента:
- BeamRiderNoFrameskip-v4
- BreakoutNoFrameskip-v4 
- EnduroNoFrameskip-v4
- PongNoFrameskip-v4

Кроме того, **если вы хотите научиться самостоятельно внедрять глубокое Q-обучение**, вам определенно следует взглянуть на эту реализацию: https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/dqn_atari.py

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/atari-envs.gif" alt="Environments" width="50%"/>

Поздравляю с окончанием этой главы!

Найдите время, чтобы по-настоящему **усвоить материал, прежде чем продолжить и попробовать дополнительные задачи**. Важно овладеть этими элементами и иметь прочную основу.

В следующем разделе **мы собираемся узнать о [Optuna](https://optuna.org/)**. Одной из наиболее важных задач в обучении с глубоким подкреплением является поиск хорошего набора обучающих гиперпараметров. А Optuna - это библиотека, которая поможет вам автоматизировать поиск.

Увидимся на бонусном блоке 2! 🔥

### Keep Learning, Stay Awesome 🤗